### Algorytmy Macierzowe lab 2

#### Konfiguracja

In [86]:
import numpy as np
import scipy.linalg 
from math import log, ceil, pow
from typing import Tuple, List

#### Generowanie macierzy

In [70]:
def generate_matrix(ncols: int, nrows: int):
    return np.random.uniform(low=0.00000001, high=1.0, size=(nrows, ncols))

#### Licznik operacji

In [62]:
class Counter:
    def __init__(self) -> None:
        self.add_counter = 0
        self.mul_counter = 0
        self.div_counter = 0
        self.sub_counter = 0
        self.operation_counter = 0

    def add(self, A: np.ndarray, B: np.ndarray) -> np.ndarray:
        self.add_counter += A.size
        self.operation_counter += A.size
        return A + B

    def mul(self, A: np.ndarray, B: np.ndarray) -> np.ndarray:
        self.mul_counter += A.size
        self.operation_counter += int(
            (A.size) ** 2.807
        )  # we assuem Strassen algorithm for multiplication
        return A @ B

    def sub(self, A: np.ndarray, B: np.ndarray) -> np.ndarray:
        self.sub_counter += A.size
        self.operation_counter += A.size
        return A - B

    def print_counters(self) -> None:
        print(f"Number of add operations: {self.add_counter}")
        print(f"Number of sub operations: {self.sub_counter}")
        print(f"Number of div operations: {self.div_counter}")
        print(f"Number of mul operations: {self.mul_counter}")
        print(f"Number of unit operations: {self.operation_counter}")

#### Funckje pomocniczne

In [63]:
def resize_matrix_to_2n(A: np.ndarray):
    """Change size of matrix to simplify processing"""
    size_A = A.shape
    new_height: int = 0
    new_width: int = 0

    new_height = find_next_power_of_2(size_A[0])

    new_width = find_next_power_of_2(size_A[1])

    new_A = np.pad(
        A, [(0, new_height - size_A[0]), (0, new_width - size_A[1])], mode="constant"
    )

    return new_A


def split(array: np.ndarray, n: int) -> Tuple:
    """Split a matrix into sub-matrices"""
    return (
        array[:n, :n],
        array[:n, n:],
        array[n:, :n],
        array[n:, n:],
    )


def find_next_power_of_2(number: int) -> int:
    """Finds closest number that is power of 2"""
    return int(pow(2, ceil(log(number) / log(2))))

#### Rekurencyjne odwracanie macierzy

In [82]:
A = generate_matrix(5, 5)


def recursive_matrix_inverse_core(A: np.ndarray, counter: Counter) -> np.ndarray:
    add = counter.add

    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0] // 2
        A11, A12, A21, A22 = split(A, split_at)

        A11_inv = recursive_matrix_inverse_core(A11, counter)
        S22 = add(A22, -mul(mul(A21, A11_inv), A12))
        S22_inv = recursive_matrix_inverse_core(S22, counter)

        B11 = mul(
            A11_inv,
            add(np.identity(A11.shape[0]), mul(mul(mul(A12, S22_inv), A21), A11_inv)),
        )
        B12 = -mul(mul(A11_inv, A12), S22_inv)
        B21 = -mul(mul(S22_inv, A21), A11_inv)
        B22 = S22_inv

        return np.concatenate(
            [np.concatenate([B11, B12], axis=1), np.concatenate([B21, B22], axis=1)],
            axis=0,
        )
    else:
        return np.full((1, 1), 1 / A[0][0])   


def recursive_matrix_inverse(A: np.ndarray, counter: Counter) -> np.ndarray:
    C = recursive_matrix_inverse_core(A, counter)
    return C


counter = Counter()
print(recursive_matrix_inverse(A.copy(), counter))
counter.print_counters()
print()
print(np.linalg.inv(A))

[[-3.9924932  -1.91729591  3.01648789  0.16134192  3.61034616]
 [ 0.70013768 -0.55858102  1.80199645 -1.88239827  1.08645359]
 [ 3.00366579  0.57126185 -0.93400563  1.01295343 -2.93299625]
 [-1.94668326  0.42035253  1.71808237 -1.4526088   1.64565262]
 [-0.41450402  0.51374963 -1.73723686  0.66359576  0.62797849]]
Number of add operations: 22
Number of sub operations: 0
Number of div operations: 0
Number of mul operations: 96
Number of unit operations: 1662

[[-3.9924932  -1.91729591  3.01648789  0.16134192  3.61034616]
 [ 0.70013768 -0.55858102  1.80199645 -1.88239827  1.08645359]
 [ 3.00366579  0.57126185 -0.93400563  1.01295343 -2.93299625]
 [-1.94668326  0.42035253  1.71808237 -1.4526088   1.64565262]
 [-0.41450402  0.51374963 -1.73723686  0.66359576  0.62797849]]


#### Rekurencyjna LU faktoryzacja

In [96]:
A = generate_matrix(5, 5)


def recursive_lu_core(
    A: np.ndarray, counter: Counter
) -> Tuple[np.ndarray, np.ndarray]:
    add = counter.add
    sub = counter.sub
    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0] // 2
        A11, A12, A21, A22 = split(A, split_at)

        L11, U11 = recursive_lu_core(A11, counter)
        U11_inv = recursive_matrix_inverse(U11, counter)

        L21 = mul(A21, U11_inv)
        L11_inv = recursive_matrix_inverse(L11, counter)

        U12 = mul(L11_inv, A12)
        L22 = add(A22, -mul(mul(mul(A21, U11_inv), L11_inv), A12))

        Ls, Us = recursive_lu_core(L22, counter)
        U22 = Us
        L22 = Us

        print("parts")
        print(L11, L21, L22)
        print()
        print(U11, U12, U22)

        return np.concatenate(
            [np.concatenate([L11, np.full(L21.shape, 0)], axis=1), np.concatenate([L21, L22], axis=1)],
            axis=0,
        ), np.concatenate(
            [np.concatenate([U11, U12], axis=1), np.concatenate([np.full(U12.shape, 0), U22], axis=1)],
            axis=0,
        )
    else:
        return np.full((1, 1), 1), np.full((1, 1), A[0, 0])


def recursive_lu(A: np.ndarray, counter: Counter) -> np.ndarray:
    C = recursive_lu_core(A, counter)
    return C


counter = Counter()
print(recursive_lu(A, counter))
counter.print_counters()
print()
print(scipy.linalg.lu(A))

parts
[[1]] [[0.16423119]] [[0.86117961]]

[[0.69093774]] [[0.38673535]] [[0.86117961]]
parts
[[1]] [[-12.38511671]] [[2.00681446]]

[[-0.03540242]] [[0.14448623]] [[2.00681446]]
parts
[[1]] [[1.44249165]
 [0.85485138]] [[-0.03540242  0.14448623]
 [ 0.          2.00681446]]

[[-0.67367827]] [[ 0.04816553 -0.38993211]] [[-0.03540242  0.14448623]
 [ 0.          2.00681446]]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 2

#### Rekurencyjny wyznacznik macierzy

In [ ]:
A = generate_matrix(2, 2)
B = generate_matrix(2, 2)


def w_core_algorithm(A: np.ndarray, B: np.ndarray, counter: Counter) -> np.ndarray:
    add = counter.add
    sub = counter.sub
    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0] // 2
        A11, A12, A21, A22 = split(A, split_at, split_at)
        B11, B12, B21, B22 = split(B, split_at, split_at)
        C_A = sub(B12, B11)
        C_D = add(A21, A22)
        A_A = w_core_algorithm(A11, B11, counter)

        U = w_core_algorithm(sub(A21, A11), sub(B12, B22), counter)
        V = w_core_algorithm(C_D, C_A, counter)
        W = add(A_A, w_core_algorithm(sub(C_D, A11), sub(B22, C_A), counter))
        W_U = add(W, U)

        C11 = add(A_A, w_core_algorithm(A12, B21, counter))
        C12 = add(add(W, V), w_core_algorithm(sub(add(A11, A12), C_D), B22, counter))
        C21 = add(W_U, w_core_algorithm(A22, sub(add(B21, C_A), B22), counter))
        C22 = add(W_U, V)

        return np.concatenate(
            [np.concatenate([C11, C12], axis=1), np.concatenate([C21, C22], axis=1)],
            axis=0,
        )
    else:
        return mul(A, B)


def winograd_algorithm(A: np.ndarray, B: np.ndarray, counter: Counter) -> np.ndarray:
    new_A, new_B = resize_matrix_to_2n(A, B)
    C = w_core_algorithm(new_A, new_B, counter)
    C = C[~np.all(C == 0, axis=1)]
    C = C[:, ~np.all(C == 0, axis=0)]
    return C


counter = Counter()
print(winograd_algorithm(A, B, counter))
counter.print_counters()
print()
print(A @ B)

#### Wykresy

In [ ]:
import matplotlib.pyplot as plt
from time import time
from matplotlib.ticker import FormatStrFormatter


times_binet: List[float] = []
flops_binet: List[int] = []

power_basis = list(range(2, 11))
powers = [2**k for k in power_basis]

for k in power_basis:
    A = np.random.rand(2**k, 2**k)
    B = np.random.rand(2**k, 2**k)

    counter = Counter()
    start_time: float = time()

    binet_algorithm(A, B, counter)

    total_time: float = time() - start_time

    times_binet.append(total_time)
    flops_binet.append(counter.operation_counter)

In [ ]:
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%d s"))
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Czas")
plt.plot(powers, times_binet)

In [ ]:
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Ilość flopsów")
plt.plot(powers, flops_binet)

In [ ]:
times_strassen: List[float] = []
flops_strassen: List[int] = []

for k in power_basis:
    A = np.random.rand(2**k, 2**k)
    B = np.random.rand(2**k, 2**k)

    counter = Counter()
    start_time: float = time()

    strassen_algorith(A, B, counter)

    total_time: float = time() - start_time

    times_strassen.append(total_time)
    flops_strassen.append(counter.operation_counter)

In [ ]:
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%d s"))
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Czas")
plt.plot(powers, times_strassen)

In [ ]:
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Ilość flopsów")
plt.plot(powers, flops_strassen)

In [ ]:
times_winograd: List[float] = []
flops_winograd: List[int] = []

for k in power_basis:
    A = np.random.rand(2**k, 2**k)
    B = np.random.rand(2**k, 2**k)

    counter = Counter()
    start_time: float = time()

    winograd_algorithm(A, B, counter)

    total_time: float = time() - start_time

    times_winograd.append(total_time)
    flops_winograd.append(counter.operation_counter)

In [ ]:
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%d s"))
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Czas")
plt.plot(powers, times_winograd)

In [ ]:
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Ilość flopsów")
plt.plot(powers, flops_winograd)

In [ ]:
A = np.array([[1, 3, 5], [2, 4, 6], [3, 5, 7]])
B = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

print(winograd_algorithm(A, B, Counter()))